## Load EPA GHG Major Emitters data from 2021 datasets (see https://www.epa.gov/ghgreporting/data-sets) for original sources

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies rooted in the S3_BUCKET : s3://redhat-osc-physical-landing-647521352890/EPA/ghgp_data_parent_company_10_2021.xlsx and s3://redhat-osc-physical-landing-647521352890/EPA/ghgp_2020_data_summary_spreadsheets/

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials and Data Commons libraries

In [1]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

import osc_ingest_trino as osc
import pyarrow as pa
import pyarrow.parquet as pq
import json

Create an S3 resource for the bucket holding source data

In [2]:
import boto3
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'epa_ghgrp'

sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': ingest_catalog,
    'schema': ingest_schema,
}

engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

# Show available schemas to ensure trino connection is set correctly
qres = engine.execute('show schemas')
qres.fetchall()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

qres = engine.execute(f'create schema if not exists {ingest_schema}')
qres.fetchall()

[(True,)]

Time for the Pandas!

In [5]:
import pandas as pd
import numpy as np
import re

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

Load EPA GHGP data file using pandas *read_excel*

In [6]:
import io

bObj = bucket.Object('EPA/ghgp_data_parent_company_10_2021.xlsx')
ghgp_bytes = io.BytesIO(bObj.get()['Body'].read())
timestamp = bObj.last_modified.isoformat()

# The source data mistakenly codes ZIP as a number, which means ZIP codes like 02134 are coded as 2134

ghgp_parent = pd.read_excel(ghgp_bytes, sheet_name=None, dtype={'FRS ID (FACILITY)':'string', 'FACILITY NAICS CODE':'string', 'PARENT CO. PERCENT OWNERSHIP':'float32'}, parse_dates=['REPORTING YEAR'], date_parser=lambda x: pd.to_datetime(x, format='%Y'), engine='openpyxl')
for year in ghgp_parent.keys():
    df = ghgp_parent[year]
    df.loc[df['PARENT COMPANY NAME']=='INTERNATIONAL PAPER CO', ['PARENT CO. STREET ADDRESS']] = '6400 Poplar Ave.'
    df.loc[df['PARENT COMPANY NAME']=='Iowa Army Ammunition Plant', ['PARENT CO. STREET ADDRESS']] = '17571 DMC Highway 79'
    df.loc[df['FACILITY NAME']=='Avon Lake Power Plant', ['FACILITY ADDRESS']] = '16157 Co Rd 22'
    df.loc[df['FACILITY NAME']=='ENCANA OIL AND GAS USA - FORT LUPTON GAS PLANT', ['FACILITY ADDRESS']] = '33570 Lake Rd'
    df['FACILITY ZIP'] = df['FACILITY ZIP'].astype(str).str.zfill(5)
    df['PARENT CO. ZIP'] = df['PARENT CO. ZIP'].astype(str).str.zfill(5)
    # print(year + ' ' + str([x for x in ghgp_parent[year]['PARENT CO. STREET ADDRESS'].to_list() if x and re.match(r'^\d+$', str(x))]))
    
    gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],f'mtiemann-GLEIF/ghgp-{year}-matches.csv')
    gleif_file.download_file(f'/tmp/ghgp-gleif.csv')
    gleif_df = pd.read_csv(f'/tmp/ghgp-gleif.csv', header=0, sep=',', dtype=str, engine='c')
    gleif_df.dropna(subset=['LEI'], inplace=True)
    gleif_dict = dict(zip(gleif_df['PARENT COMPANY NAME'], gleif_df.LEI))
    
    df['LEI'] = df['PARENT COMPANY NAME'].map(gleif_dict)
    print(f'year = {year}; non-NULL LEIs = {len(df[df.LEI.notnull()])}')
    
    cols = df.columns.tolist()
    ghgp_parent[year] = df[cols[0:2] + [cols[-1]] + cols[2:-1]]
    osc.enforce_sql_column_names(ghgp_parent[year], inplace=True)

year = 2020; non-NULL LEIs = 6699
year = 2019; non-NULL LEIs = 6865
year = 2018; non-NULL LEIs = 6873
year = 2017; non-NULL LEIs = 6860
year = 2016; non-NULL LEIs = 6955
year = 2015; non-NULL LEIs = 7220
year = 2014; non-NULL LEIs = 7054
year = 2013; non-NULL LEIs = 6880
year = 2012; non-NULL LEIs = 6723
year = 2011; non-NULL LEIs = 6296
year = 2010; non-NULL LEIs = 5013


In [7]:
# As a next step, load per-year data, which is more detailed
# i.e., bObj = bucket.Object(f'EPA/ghgp_2020_data_summary_spreadsheets/ghgp_data_{year}.xlsx')

bObj = bucket.Object(f'EPA/ghgp_2020_data_summary_spreadsheets/ghgp_data_by_year.xlsx')
ghgp_bytes = io.BytesIO(bObj.get()['Body'].read())
timestamp = max(timestamp, bObj.last_modified.isoformat())
    
ghg_data = pd.read_excel(ghgp_bytes, sheet_name=None, dtype={'FRS Id':'string'}, skiprows=3, engine='openpyxl')
for df_name, df in ghg_data.items():
    if df_name=='Industry Type':
        break
    if 'Zip Code' in df:
        df['Zip Code'] = df['Zip Code'].astype(str).str.zfill(5)
    elif 'Reported Zip Code' in df:
        df['Reported Zip Code'] = df['Reported Zip Code'].astype(str).str.zfill(5)
    df['Primary NAICS Code'] = df['Primary NAICS Code'].astype(str)
    df.replace('confidential', -1, inplace=True)
    if df_name=='CO2 Injection':
        # All info essentially confidential, but at least we know where facilities are
        ghg_data[df_name] = df
        continue
    df = df.convert_dtypes()
    # Some numeric info marked 'confidential' which will make for interesting NA handling...
    # df.info(verbose=True)
    ghg_data[df_name] = df

Construct the combined metadata by merging existing table metadata and custom metadata.
Note: The metadata content must be JSON serialisable and encoded as bytes; the metadata key must also be encoded as bytes.

In [8]:
tablename = 'parent_company'
df = pd.concat(ghgp_parent, ignore_index=True).convert_dtypes()
osc.drop_unmanaged_table(ingest_catalog, ingest_schema, tablename, engine, trino_bucket)
osc.ingest_unmanaged_parquet(df, ingest_schema, tablename, trino_bucket)
tabledef = osc.unmanaged_parquet_tabledef(df, ingest_catalog, ingest_schema, tablename, trino_bucket,
                                         typemap={'datetime64[ns]':'timestamp(3)'})
qres = engine.execute(tabledef)
qres.fetchall()

[(True,)]

### Melt the generation data into a more tidy format, dropping NA values

In [9]:
for tablename, ghg_table in ghg_data.items():
    if tablename=='Industry Type':
        break
    print(f':{tablename}:')
    # Melt the data...
    if tablename in ['Gathering & Boosting', 'Transmission Pipelines', 'Geologic Sequestration of CO2']:
        ghg_value_vars = ghg_table.columns[-5:]
        ghg_id_vars = ghg_table.columns[:-5]
    elif tablename=='Suppliers':
        continue
    else:
        ghg_value_vars = ghg_table.columns[-10:]
        ghg_id_vars = ghg_table.columns[:-10]
    
    # We leave in place the fact that all total reported emissions retain their categorization as to source
    # It's temping to rename these all to 'total_reported_emissions' so that they'd magically sum together if asked.
    # But there's no easy way in SQL to do that join without the tables exploding (because there's no natural key)
    _, value_name = ghg_value_vars[0].split(' ', 1)
    # value_name = 'total_reported_emissions'
    
    ghg_melted_df = ghg_table.melt(ghg_id_vars, ghg_value_vars, var_name='year', value_name=value_name)
    ghg_melted_df.dropna(subset=[value_name],inplace=True)
    ghg_melted_df.year = ghg_melted_df.year.apply(lambda x: x.split(' ', 1)[0])
    ghg_melted_df['year'] = pd.to_datetime(ghg_melted_df['year'], format='%Y', utc=True)
    ghg_melted_df[value_name] = ghg_melted_df[value_name].astype('float64')
    ghg_melted_df = ghg_melted_df.convert_dtypes()
    # Put year at the end to make for more friendly partitioning
    ghg_melted_df = ghg_melted_df[ghg_melted_df.columns[:-2].to_list() + [ghg_melted_df.columns[-1], ghg_melted_df.columns[-2]]]
    tablename = osc.sql_compliant_name(tablename)
    osc.enforce_sql_column_names(ghg_melted_df, inplace=True)
    osc.drop_unmanaged_table(ingest_catalog, ingest_schema, tablename, engine, trino_bucket)
    osc.ingest_unmanaged_parquet(ghg_melted_df, ingest_schema, tablename, trino_bucket)
    tabledef = osc.unmanaged_parquet_tabledef(ghg_melted_df, ingest_catalog, ingest_schema, tablename, trino_bucket)
    qres = engine.execute(tabledef)
    qres.fetchall()

:Direct Emitters:
:Onshore Oil & Gas Prod.:
:Gathering & Boosting:
:Transmission Pipelines:
:LDC - Direct Emissions:
:SF6 from Elec. Equip.:
:Suppliers:
:CO2 Injection:
:Geologic Sequestration of CO2:


In [10]:
tablename = 'Suppliers'
ghg_table = ghg_data[tablename].copy()
ghg_table['Zip Code'] = ghg_table['Zip Code'].astype(str).str.zfill(5)

ghg_value_names = [':'.join(list(reversed(x.split(' ', 1)))) for x in ghg_table.columns[-70:]]
ghg_table = ghg_table.rename(columns=dict(zip(ghg_table.columns[-70:], ghg_value_names)))
ghg_id_vars = ghg_table.columns[:-70]
# print(ghg_id_vars)
# print(ghg_value_names)
stubnames = [x.split(':', 1)[0] for x in ghg_value_names[0:70:10]]
# print(stubnames)
suppliers_df = pd.wide_to_long(ghg_table, stubnames=stubnames, i=ghg_id_vars, j='year', sep=':')
# Take care to null out all the `confidential` data
for sn in stubnames:
    suppliers_df[sn] = suppliers_df[sn].astype('Float64')
suppliers_df.dropna(subset=stubnames, how='all', inplace=True)
suppliers_df.reset_index(inplace=True)
suppliers_df.loc[suppliers_df.year.notnull(), 'year'] = pd.to_datetime(suppliers_df.year, format='%Y', utc=True)
suppliers_df = suppliers_df.convert_dtypes()
# Put year at the end to make for more friendly partitioning
year_index = suppliers_df.columns.get_loc('year')
suppliers_df = suppliers_df[suppliers_df.columns[:year_index].to_list()
                            + suppliers_df.columns[year_index+1:].to_list()
                            + [suppliers_df.columns[year_index]]]

for sn in stubnames:
    new_stubname = ' '.join(sn.split(' ')[4:] + ['ghg'])
    suppliers_df.rename(columns={sn:new_stubname}, inplace=True)

tablename = osc.sql_compliant_name(tablename)
osc.enforce_sql_column_names(suppliers_df, inplace=True)
osc.drop_unmanaged_table(ingest_catalog, ingest_schema, tablename, engine, trino_bucket)
osc.ingest_unmanaged_parquet(suppliers_df, ingest_schema, tablename, trino_bucket)
tabledef = osc.unmanaged_parquet_tabledef(suppliers_df, ingest_catalog, ingest_schema, tablename, trino_bucket)
qres = engine.execute(tabledef)
qres.fetchall()

[(True,)]

In [11]:
# Everything below here is speculative / in process of design

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*